In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

In [2]:
ovr_db = pd.read_excel(r"C:\Users\n740789\Documents\Projects_local\DataSets\overrides\202502_BBDD_Overrides.xlsx",
                   sheet_name="OVR DB")
map_df = pd.read_excel(r"C:\Users\n740789\Documents\Projects_local\DataSets\overrides\202502_BBDD_Overrides.xlsx",
                   sheet_name="MAPEO P-S")

In [3]:
# read latest datafeed at the issuer level
target_columns = [
    "issuer_name",
    "permid",
    "clarityid",
    "company_inheriting",
    "parent_company",
    "str_001_s",
    "str_002_ec",
    "str_003_ec",
    "str_003b_ec",
    "str_004_asec",
    "str_005_ec",
    "cs_001_sec",
    "cs_003_sec",
    "cs_002_ec",
    "str_006_sec",
    "art_8_basicos"
]

datafeed = pd.read_csv(
    r"C:\Users\n740789\Documents\Projects_local\DataSets\DATAFEED\ficheros_tratados\2025\20250201_Equities_feed_IssuerLevel_sinOVR.csv",
    usecols=target_columns,
    dtype={"permid": str, "clarityid": str}
)

In [4]:
# rename column names

col_dict = {
    "str_001_s" : "str_001",
    "str_002_ec" : "str_002",
    "str_003_ec" : "str_003",
    "str_003b_ec" : "str_003b",
    "str_004_asec" : "str_004",
    "str_005_ec" : "str_005",
    "cs_001_sec" : "cs_001",
    "cs_003_sec" : "cs_003",
    "cs_002_ec" : "cs_002",
    "str_006_sec" : "str_006",
    "art_8_basicos" : "art_8" 
}

datafeed.rename(columns=col_dict, inplace=True)

In [5]:
ovr_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 69 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   #                                            750 non-null    int64  
 1   Fecha aplicación OVR                         750 non-null    object 
 2   Fecha última revisión                        750 non-null    object 
 3   Próxima revisión                             748 non-null    object 
 4   ClarityID                                    750 non-null    int64  
 5   permId                                       750 non-null    int64  
 6   Sustainalytics ID                            750 non-null    object 
 7   IssuerID                                     747 non-null    object 
 8   Issuer Name                                  750 non-null    object 
 9   SNT-WORLD
(Y/-)                              735 non-null    object 
 10  PA

In [6]:
map_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7757 entries, 0 to 7756
Data columns (total 31 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   #                                  7757 non-null   int64         
 1   ClarityID                          7757 non-null   int64         
 2   permId (Clarity)                   7757 non-null   object        
 3   IssuerID                           2509 non-null   object        
 4   Issuer Name                        7757 non-null   object        
 5   SNT-WORLD
(Y/-)                    7675 non-null   object        
 6   PARENT NAME (Aladdin)              7688 non-null   object        
 7   PARENT IssuerID (Aladdin)          7685 non-null   object        
 8   Parent (P) / Subsidiaria (S)       7757 non-null   object        
 9   PARENT NAME (Clarity)              7755 non-null   object        
 10  Parent ClarityID                   7

In [ ]:
for c in datafeed.columns:
    print(c)

In [7]:
# stadarise spelling of columns
ovr_db.columns = ovr_db.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('\n','_').str.replace("*","")
map_df.columns = map_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('\n','_').str.replace("*","")

# rename columns
map_df.rename(columns={
    'permid_clarity':'permid',
    'snt-world_y/-':'snt_world',
    }, inplace=True)

ovr_db.rename(columns={
    'fecha_aplicación_ovr':'ovr_date',
    'fecha_última_revisión':'last_check_date',
    'próxima_revisión':'next_check_date',
    'snt-world_y/-':'snt_world',
}, inplace=True)


ovr_col_dict = {
    "ovr_str001_sec" : "ovr_str001",
    "ovr_str002_ec" : "ovr_str002",
    "ovr_str003_ec" : "ovr_str003",
    "ovr_str003b_ec" : "ovr_str003b",
    "ovr_str004_sec" : "ovr_str004",
    "ovr_str005_ec" : "ovr_str005",
    "ovr_str006_sec" : "ovr_str006",
    "ovr_articulo_8_c" : "ovr_art_8",
    "ovr_cs001_sec" : "ovr_cs001",
    "ovr_cs002_ec" : "ovr_cs002",
    "ovr_cs003_sec" : "ovr_cs003"
}

# rename targe ovr col name
ovr_db.rename(columns=ovr_col_dict, inplace=True)
map_df.rename(columns=ovr_col_dict, inplace=True)

# make sure column permid is str
ovr_db['permid'] = ovr_db['permid'].astype(str)
ovr_db['clarityid'] = ovr_db['clarityid'].astype(str)
map_df['clarityid'] = map_df['clarityid'].astype(str)
map_df['permid'] = map_df['permid'].astype(str)

# parse ovr_date, last_check_date, next_check_date to datetime objects
ovr_db['ovr_date'] = pd.to_datetime(ovr_db['ovr_date'], errors='coerce')
ovr_db['last_check_date'] = pd.to_datetime(ovr_db['last_check_date'], errors='coerce')
ovr_db['next_check_date'] = pd.to_datetime(ovr_db['next_check_date'], errors='coerce') 

C:\Users\n740789\AppData\Local\Temp\ipykernel_23828\3909354322.py:44: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ovr_db['ovr_date'] = pd.to_datetime(ovr_db['ovr_date'], errors='coerce')


Remove unecesary columns and generate fileterd dfs

In [8]:
# let's get rig of str_007
# drop columns with indices 38,39, 40
ovr_db_filtered = ovr_db.drop(ovr_db.columns[[38, 39, 40]], axis=1).copy()
# drop from map_df column number 22
map_df = map_df.drop(map_df.columns[22], axis=1).copy()

In [9]:
# List of columns to process
ovr_col_list = [
    "ovr_str001",
    "ovr_str002",
    "ovr_str003",
    "ovr_str003b",
    "ovr_str004",
    "ovr_str005",
    "ovr_str006",
    "ovr_art_8",
    "ovr_cs001",
    "ovr_cs002",
    "ovr_cs003"
]

col_list_deflag = [
    "str_001",
    "str_002",
    "str_003",
    "str_003b",
    "str_004",
    "str_005",
    "str_006",
    "art_8",
    "cs_001",
    "cs_003",
    "cs_002",
]

# Let's get rid of flags
# Function to replace 'FLAG' with NaN in specific columns
#def deflagg_df(df, col_list):
#    for col in col_list:
#        if col in df.columns:  # Check if the column exists in the DataFrame
#            df[col] = df[col].replace('FLAG', 'OK') # Replace 'FLAG' with 'OK'
#    return df


## Apply the function to the DataFrame
#df_filtered = deflagg_df(df_filtered.copy(), ovr_col_list)
#map_df = deflagg_df(map_df.copy(), ovr_col_list)
#datafeed = deflagg_df(datafeed.copy(), col_list_deflag)

In [10]:
# Fix override grounds
def fix_spell(ground: str) -> str:
    if ground in ['BIC', 'BiC']:
        return 'SR'
    elif ground == 'Exposición':
        return 'exposure'
    elif ground in ['Inheritance', 'Inheritance de subsidiaria', 'Inheritance por caida de SR']:
        return 'inheritance'
    elif ground == 'Comité':
        return 'committee'
    elif ground in ['Controversia', 'Fallo controversias Clarity', 'OVR por controversia nivel 3/5 en Susta pero en Clarity 1/100']:
        return 'controversy'
    else:
        return ground  # Return the original input if no conditions match

map_df.motivo_principal_ovr = map_df.motivo_principal_ovr.apply(fix_spell)


# filter out rows with motivo_principal_ovr == 'Inheritance'
#map_filtered = map_df[map_df['motivo_principal_ovr']!='Inheritance'].copy()

# filter out rows with motivo_principal_ovr == 'committee'
comitte_df = map_df[map_df.motivo_principal_ovr == 'committee']

Display results

In [ ]:
print(f"There are {len(ovr_db.permid.unique())} unique permids while the df has {len(ovr_db)} rows")
print(f"There are {len(map_df.permid.unique())} unique permids while the map_df has {len(map_df)} rows")

print("display overrides by year month")
print(ovr_db.ovr_date.dt.strftime('%Y_%m').value_counts(dropna=False).sort_index())


print("display overrides by strategy")

dfs_list = [ovr_db, map_df] # other dfs df_deflagged, df_filtered

for dfs in dfs_list:
    for col in ovr_col_list:
        print(dfs[col].value_counts(dropna=True, normalize=True).round(2), '\n')


In [11]:
# merge map_df with datafeed on permid
map_datafeed = pd.merge(map_df, datafeed, on='permid', how='left')


In [12]:
# merge df_filtered with datafeed on permid
ovr_db_datafeed = pd.merge(ovr_db_filtered, datafeed, on='clarityid', how='left')

In [ ]:
print(map_datafeed.shape[0])
print(ovr_db_datafeed.shape[0])

In [ ]:
ovr_db_datafeed.info()

In [13]:
def drop_columns_by_prefix(df, prefixes):
    columns_to_drop = [col for col in df.columns if any(col.startswith(prefix) for prefix in prefixes)]
    return df.drop(columns=columns_to_drop)

# Drop columns starting with "original_" or "final_"
prefixes_to_remove = ["original_", "final_"]
ovr_db_datafeed = drop_columns_by_prefix(ovr_db_datafeed, prefixes_to_remove)

In [18]:
#rename column permid_x and issuer_name_x df_datafeed to permid and issuer_name
ovr_db_datafeed.rename(columns={
    'permid_x':'permid',
    'issuer_name_x':'issuer_name'
}, inplace=True)
#rename column permid_x and issuer_name_x map_datafeed to permid and issuer_name
map_datafeed.rename(columns={
    'clarityid_x':'clarityid',
    'issuer_name_x':'issuer_name'
}, inplace=True)


result_target_columns = [
    "last_check_date",
    "next_check_date",
    'permid',
    "clarityid",
    'issuer_name',
    "company_inheriting",
    "parent_company",
    "ovr_str001",
    "ovr_str002",
    "ovr_str003",
    "ovr_str003b",
    "ovr_str004",
    "ovr_str005",
    "ovr_str006",
    "ovr_cs001",
    "ovr_cs002",
    "ovr_cs003",
    "ovr_art_8",
    "str_001",
    "str_002",
    "str_003",
    "str_003b",
    "str_004",
    "str_005",
    "str_006",
    "art_8",
    "cs_001",
    "cs_003",
    "cs_002"
]
result_target_columns_map = [
    'permid',
    "clarityid",
    'issuer_name',
    "company_inheriting",
    "parent_company",
    "ovr_str001",
    "ovr_str002",
    "ovr_str003",
    "ovr_str003b",
    "ovr_str004",
    "ovr_str005",
    "ovr_str006",
    "ovr_cs001",
    "ovr_cs002",
    "ovr_cs003",
    "ovr_art_8",
    "str_001",
    "str_002",
    "str_003",
    "str_003b",
    "str_004",
    "str_005",
    "str_006",
    "art_8",
    "cs_001",
    "cs_003",
    "cs_002"
]

# filter df_datafeed to only include the columns in result_target_columns
result_df = ovr_db_datafeed[result_target_columns].copy()
# filter map_datafeed to only include the columns in result_target_columns
result_map_df = map_datafeed[result_target_columns_map].copy()

In [ ]:
result_map_df.info()

In [20]:
def active_ovr(row, column_pairs):
    result = {}
    for ovr_col, base_col in column_pairs:
        if pd.isna(row[ovr_col]) or pd.isna(row[base_col]):
            result[f"{ovr_col}_active"] = np.nan
        else:
            result[f"{ovr_col}_active"] = row[ovr_col] != row[base_col]
    return result

In [ ]:
# Define column pairs to compare based on the provided columns
column_pairs_to_compare = [
    ("ovr_str001", "str_001"),
    ("ovr_str002", "str_002"),
    ("ovr_str003", "str_003"),
    ("ovr_str003b", "str_003b"),
    ("ovr_str004", "str_004"),
    ("ovr_str005", "str_005"),
    ("ovr_str006", "str_006"),
    ("ovr_cs001", "cs_001"),
    ("ovr_cs002", "cs_002"),
    ("ovr_cs003", "cs_003"),
    ("ovr_art_8", "art_8"),
]

# Add new columns for each override
for index, row in result_df.iterrows():
    active_overrides = active_ovr(row, column_pairs_to_compare)
    for col, value in active_overrides.items():
        result_df.at[index, col] = value

In [ ]:
# Define column pairs to compare based on the provided columns
column_pairs_to_compare = [
    ("ovr_str001", "str_001"),
    ("ovr_str002", "str_002"),
    ("ovr_str003", "str_003"),
    ("ovr_str003b", "str_003b"),
    ("ovr_str004", "str_004"),
    ("ovr_str005", "str_005"),
    ("ovr_str006", "str_006"),
    ("ovr_cs001", "cs_001"),
    ("ovr_cs002", "cs_002"),
    ("ovr_cs003", "cs_003"),
    ("ovr_art_8", "art_8"),
]

# Add new columns for each override 
for index, row in result_map_df.iterrows():
    active_overrides = active_ovr(row, column_pairs_to_compare)
    for col, value in active_overrides.items():
        result_map_df.at[index, col] = value


In [ ]:
result_map_df.ovr_str002_active.value_counts()

In [ ]:
"last_check_date",
"next_check_date",
"clarityid",
"company_inheriting",
"parent_company",
"ovr_str001",
"ovr_str002",
"ovr_str003",
"ovr_str003b",
"ovr_str004",
"ovr_str005",
"ovr_str006",
"ovr_cs001",
"ovr_cs002",
"ovr_cs003",
"ovr_art_8",
"str_001",
"str_002",
"str_003",
"str_003b",
"str_004",
"str_005",
"cs_001",
"cs_003",
"cs_002",
"str_006",
"art_8",
"ovr_str001_active",
"ovr_str002_active",
"ovr_str003_active",
"ovr_str003b_active",
"ovr_str004_active",
"ovr_str005_active",
"ovr_str006_active",
"ovr_cs001_active",
"ovr_cs002_active",
"ovr_cs003_active",
"ovr_art_8_active",

In [25]:
def save_result_df_to_excel(result_df, output_file, basic_columns):
    output_dir = os.path.dirname(output_file)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    strategy_columns = [
        "str_001", "str_002", "str_003", "str_003b", 
        "str_004", "str_005","str_006", "art_8",
        "cs_001", "cs_002", "cs_003" 
    ]

    ovr_columns = [
        "ovr_str001", "ovr_str002", "ovr_str003", "ovr_str003b", 
        "ovr_str004", "ovr_str005", "ovr_str006", "ovr_art_8",
        "ovr_cs001", "ovr_cs002", "ovr_cs003"
    ]

    ovr_active_columns = [
        "ovr_str001_active", "ovr_str002_active", "ovr_str003_active", "ovr_str003b_active",
        "ovr_str004_active", "ovr_str005_active", "ovr_str006_active", "ovr_art_8_active",
        "ovr_cs001_active", "ovr_cs002_active", "ovr_cs003_active", 
    ]

    with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
        result_df.to_excel(writer, sheet_name="Full_Data", index=False)

        for strategy, ovr_col, active_col in zip(strategy_columns, ovr_columns, ovr_active_columns):
            relevant_columns = basic_columns + [
                col for col in [strategy, ovr_col, active_col] if col in result_df.columns
            ]

            if relevant_columns:
                result_df[relevant_columns].to_excel(writer, sheet_name=f"{strategy}", index=False)

    print(f"Data has been successfully saved to {output_file}")



In [ ]:
basic_columns = [
    "last_check_date",
    "next_check_date",
    "permid",
    "clarityid",
    "issuer_name",
    "company_inheriting",
    "parent_company",
]
save_result_df_to_excel(result_df, r"output\20250201_override_analysis.xlsx", basic_columns=basic_columns)

In [ ]:
result_df.head()

In [ ]:
basic_columns = [
    "permid",
    "clarityid",
    "issuer_name",
    "company_inheriting",
    "parent_company",
]
save_result_df_to_excel(result_map_df, r"output\20250201_override_analysis_mapsheet.xlsx", basic_columns=basic_columns)

In [ ]:
for col in result_map_df.columns:
    if col.endswith("_active"):
        print(result_map_df[col].value_counts(dropna=True))
        print("\n")

In [ ]:
for col in result_df.columns:
    if col.endswith("_active"):
        print(result_df[col].value_counts(dropna=True))
        print("\n")

In [ ]:
# Mapping of PostgreSQL data types to Apache Hive data types
type_mapping = {
    "character varying": "STRING",
    "boolean": "BOOLEAN",
    "integer": "INT",
    "bigint": "BIGINT",
    "double precision": "DOUBLE"
}

# Input: Column names and data types
input_data = """
 isin                                               | character varying
 instrument_type                                    | character varying
 issuer_name                                        | character varying
 issuer_country                                     | character varying
 gics2_industry                                     | character varying
 region                                             | character varying
 company_inheriting                                 | boolean
 parent_company                                     | character varying
 esg_score                                          | integer
 esg_score_relevance                                | double precision
 e_score                                            | double precision
 s_score                                            | integer
 g_score                                            | integer
 esg_rating                                         | character varying
 sustainability_rating                              | character varying
 global_sustainability_rating                       | character varying
 controv_access_to_basic_services                   | double precision
 critical_controv_access_to_basic_services          | boolean
 controv_accounting_and_taxation                    | double precision
 critical_controv_accounting_and_taxation           | boolean
 controv_animal_welfare                             | double precision
 critical_controv_animal_welfare                    | boolean
 controv_animal_welfare_sc                          | double precision
 critical_controv_animal_welfare_sc                 | boolean
 controv_anticompetitive_practices                  | double precision
 critical_controv_anticompetitive_practices         | boolean
 controv_bribery_and_corruption                     | double precision
 critical_controv_bribery_and_corruption            | boolean
 controv_bribery_and_corruption_sc                  | double precision
 critical_controv_bribery_and_corruption_sc         | boolean
 controv_business_ethics                            | double precision
 critical_controv_business_ethics                   | boolean
 controv_business_ethics_sc                         | double precision
 critical_controv_business_ethics_sc                | boolean
 controv_carbon_impact_of_products                  | double precision
 critical_controv_carbon_impact_of_products         | boolean
 controv_community_relations                        | double precision
 critical_controv_community_relations               | boolean
 controv_community_relations_sc                     | double precision
 critical_controv_community_relations_sc            | boolean
 controv_corporate_governance                       | double precision
 critical_controv_corporate_governance              | boolean
 controv_data_privacy_and_security                  | double precision
 critical_controv_data_privacy_and_security         | boolean
 controv_emissions_effluents_and_waste              | double precision
 critical_controv_emissions_effluents_and_waste     | boolean
 controv_emissions_effluents_and_waste_sc           | double precision
 critical_controv_emissions_effluents_and_waste_sc  | boolean
 controv_employees_human_rights                     | double precision
 critical_controv_employees_human_rights            | boolean
 controv_employees_human_rights_sc                  | double precision
 critical_controv_employees_human_rights_sc         | boolean
 controv_energy_use_and_ghg_emissions               | double precision
 critical_controv_energy_use_and_ghg_emissions      | boolean
 controv_energy_use_and_ghg_emissions_sc            | double precision
 critical_controv_energy_use_and_ghg_emissions_sc   | boolean
 controv_environmental_impact_of_products           | double precision
 critical_controv_environmental_impact_of_products  | boolean
 controv_intellectual_property                      | double precision
 critical_controv_intellectual_property             | boolean
 controv_labour_relations                           | double precision
 critical_controv_labour_relations                  | boolean
 controv_labour_relations_sc                        | double precision
 critical_controv_labour_relations_sc               | boolean
 controv_land_use_and_biodiversity                  | double precision
 critical_controv_land_use_and_biodiversity         | boolean
 controv_land_use_and_biodiversity_sc               | double precision
 critical_controv_land_use_and_biodiversity_sc      | boolean
 controv_lobbying_and_public_policy                 | double precision
 critical_controv_lobbying_and_public_policy        | boolean
 controv_marketing_practices                        | double precision
 critical_controv_marketing_practices               | boolean
 controv_media_ethics                               | double precision
 critical_controv_media_ethics                      | boolean
 controv_occupational_health_and_safety             | double precision
 critical_controv_occupational_health_and_safety    | boolean
 controv_occupational_health_and_safety_sc          | double precision
 critical_controv_occupational_health_and_safety_sc | boolean
 controv_quality_and_safety                         | double precision
 critical_controv_quality_and_safety                | boolean
 controv_sanctions                                  | double precision
 critical_controv_sanctions                         | boolean
 controv_social_impact_of_products                  | double precision
 critical_controv_social_impact_of_products         | boolean
 controv_society_human_rights                       | double precision
 critical_controv_society_human_rights              | boolean
 controv_society_human_rights_sc                    | double precision
 critical_controv_society_human_rights_sc           | boolean
 controv_water_use                                  | double precision
 critical_controv_water_use                         | boolean
 controv_water_use_sc                               | double precision
 critical_controv_water_use_sc                      | boolean
 controv_weapons                                    | double precision
 critical_controv_weapons                           | boolean
 any_critical_controversy                           | boolean
 exp_abortifacents_part                             | boolean
 minexp_abortifacents_part                          | double precision
 maxexp_abortifacents_part                          | double precision
 exp_agrochemical_products_part                     | boolean
 minexp_agrochemical_products_part                  | double precision
 maxexp_agrochemical_products_part                  | double precision
 exp_agrochemical_products_prod                     | boolean
 minexp_agrochemical_products_prod                  | double precision
 maxexp_agrochemical_products_prod                  | double precision
 exp_alcohol_part                                   | boolean
 minexp_alcohol_part                                | double precision
 maxexp_alcohol_part                                | double precision
 exp_alchol_prod                                    | boolean
 minexp_alchol_prod                                 | double precision
 maxexp_alchol_prod                                 | double precision
 exp_animal_testing_part                            | boolean
 minexp_animal_testing_part                         | double precision
 maxexp_animal_testing_part                         | double precision
 exp_animal_testing_cosmetic_part                   | boolean
 minexp_animal_testing_cosmetic_part                | double precision
 maxexp_animal_testing_cosmetic_part                | double precision
 exp_antip_landmines_part                           | boolean
 minexp_antip_landmines_part                        | double precision
 maxexp_antip_landmines_part                        | double precision
 exp_armament_part                                  | boolean
 minexp_armament_part                               | double precision
 maxexp_armament_part                               | double precision
 exp_armament_prod                                  | boolean
 minexp_armament_prod                               | double precision
 maxexp_armament_prod                               | double precision
 exp_chemic_biolog_weapons_part                     | boolean
 minexp_chemic_biolog_weapons_part                  | double precision
 maxexp_chemic_biolog_weapons_part                  | double precision
 exp_cluster_bombs_part                             | boolean
 minexp_cluster_bombs_part                          | double precision
 maxexp_cluster_bombs_part                          | double precision
 exp_coal_mining_part                               | boolean
 minexp_coal_mining_part                            | double precision
 maxexp_coal_mining_part                            | double precision
 exp_coal_mining_prod                               | boolean
 minexp_coal_mining_prod                            | double precision
 maxexp_coal_mining_prod                            | double precision
 exp_coal_power_gen_part                            | boolean
 minexp_coal_power_gen_part                         | double precision
 maxexp_coal_power_gen_part                         | double precision
 exp_coal_power_gen_prod                            | boolean
 minexp_coal_power_gen_prod                         | double precision
 maxexp_coal_power_gen_prod                         | double precision
 exp_contraceptives_part                            | boolean
 minexp_contraceptives_part                         | double precision
 maxexp_contraceptives_part                         | double precision
 exp_depleted_uranium_part                          | boolean
 minexp_depleted_uranium_part                       | double precision
 maxexp_depleted_uranium_part                       | double precision
 exp_embryonic_stem_cell_research_part              | boolean
 minexp_embryonic_stem_cell_research_part           | double precision
 maxexp_embryonic_stem_cell_research_part           | double precision
 exp_fossil_fuels_part                              | boolean
 minexp_fossil_fuels_part                           | double precision
 maxexp_fossil_fuels_part                           | double precision
 exp_fossil_fuels_prod                              | boolean
 minexp_fossil_fuels_prod                           | double precision
 maxexp_fossil_fuels_prod                           | double precision
 exp_fur_exotic_leather_part                        | boolean
 minexp_fur_exotic_leather_part                     | double precision
 maxexp_fur_exotic_leather_part                     | double precision
 exp_gambling_part                                  | boolean
 minexp_gambling_part                               | double precision
 maxexp_gambling_part                               | double precision
 exp_gambling_prod                                  | boolean
 minexp_gambling_prod                               | double precision
 maxexp_gambling_prod                               | double precision
 exp_gmo_products_part                              | boolean
 minexp_gmo_products_part                           | double precision
 maxexp_gmo_products_part                           | double precision
 exp_gmo_research_part                              | boolean
 minexp_gmo_research_part                           | double precision
 maxexp_gmo_research_part                           | double precision
 exp_meat_products_part                             | boolean
 minexp_meat_products_part                          | double precision
 maxexp_meat_products_part                          | double precision
 exp_meat_products_prod                             | boolean
 minexp_meat_products_prod                          | double precision
 maxexp_meat_products_prod                          | double precision
 exp_cannabis_part                                  | boolean
 minexp_cannabis_part                               | double precision
 maxexp_cannabis_part                               | double precision
 exp_nuclear_energy_part                            | boolean
 minexp_nuclear_energy_part                         | double precision
 maxexp_nuclear_energy_part                         | double precision
 exp_nuclear_energy_prod                            | boolean
 minexp_nuclear_energy_prod                         | double precision
 maxexp_nuclear_energy_prod                         | double precision
 exp_nuclear_weapons_part                           | boolean
 minexp_nuclear_weapons_part                        | double precision
 maxexp_nuclear_weapons_part                        | double precision
 exp_oil_sands_part                                 | boolean
 minexp_oil_sands_part                              | double precision
 maxexp_oil_sands_part                              | double precision
 exp_oil_sands_prod                                 | boolean
 minexp_oil_sands_prod                              | double precision
 maxexp_oil_sands_prod                              | double precision
 exp_palm_oil_part                                  | boolean
 minexp_palm_oil_part                               | double precision
 maxexp_palm_oil_part                               | double precision
 exp_pork_products_part                             | boolean
 minexp_pork_products_part                          | double precision
 maxexp_pork_products_part                          | double precision
 exp_pork_products_prod                             | boolean
 minexp_pork_products_prod                          | double precision
 maxexp_pork_products_prod                          | double precision
 exp_pornography_part                               | boolean
 minexp_pornography_part                            | double precision
 maxexp_pornography_part                            | double precision
 exp_shale_energy_part                              | boolean
 minexp_shale_energy_part                           | double precision
 maxexp_shale_energy_part                           | double precision
 exp_shale_energy_prod                              | boolean
 minexp_shale_energy_prod                           | double precision
 maxexp_shale_energy_prod                           | double precision
 exp_stem_cell_research                             | boolean
 minexp_stem_cell_research                          | double precision
 maxexp_stem_cell_research                          | double precision
 exp_tobacco_part                                   | boolean
 minexp_tobacco_part                                | double precision
 maxexp_tobacco_part                                | double precision
 exp_tobacco_prod                                   | boolean
 minexp_tobacco_prod                                | double precision
 maxexp_tobacco_prod                                | double precision
 exp_artic_oil_part                                 | boolean
 minexp_artic_oil_part                              | double precision
 maxexp_artic_oil_part                              | double precision
 exp_predatory_lending_part                         | boolean
 minexp_predatory_lending_part                      | double precision
 maxexp_predatory_lending_part                      | double precision
 exp_small_arms_part                                | boolean
 minexp_small_arms_part                             | double precision
 maxexp_small_arms_part                             | double precision
 exp_white_phosporus_part                           | boolean
 minexp_white_phosporus_part                        | double precision
 maxexp_white_phosporus_part                        | double precision
 str_001_s                                          | character varying
 str_002_ec                                         | character varying
 str_003_ec                                         | character varying
 str_004_asec                                       | character varying
 clarityid                                          | bigint
 str_005_ec                                         | character varying
 cs_001_sec                                         | character varying
 gp_esccp                                           | character varying
 cs_003_sec                                         | character varying
 cs_002_ec                                          | character varying
 str_006_sec                                        | character varying
 str_007_sect                                       | character varying
 gp_esccp_22                                        | character varying
 gp_esccp_25                                        | character varying
 gp_esccp_30                                        | character varying
 art_8_basicos                                      | character varying
 permid                                             | bigint
 countrycode                                        | character varying
 str_003b_ec                                        | character varying
 exp_nuclear_weapons_prod                           | boolean
 minexp_nuclear_weapons_prod                        | double precision
 maxexp_nuclear_weapons_prod                        | double precision
 exp_cluster_bombs_prod                             | boolean
 minexp_cluster_bombs_prod                          | double precision
 maxexp_cluster_bombs_prod                          | double precision
 exp_thermal_coal_prod                              | boolean
 minexp_thermal_coal_prod                           | double precision
 maxexp_thermal_coal_prod                           | double precision
 exp_gas_fuels_prod                                 | boolean
 minexp_gas_fuels_prod                              | double precision
 maxexp_gas_fuels_prod                              | double precision
 exp_gas_fuels_part                                 | boolean
 minexp_gas_fuels_part                              | double precision
 maxexp_gas_fuels_part                              | double precision
 exp_oil_fuels_prod                                 | boolean
 minexp_oil_fuels_prod                              | double precision
 maxexp_oil_fuels_prod                              | double precision
 exp_oil_fuels_part                                 | boolean
 minexp_oil_fuels_part                              | double precision
 maxexp_oil_fuels_part                              | double precision
 datafeed_date                                      | integer

"""

# Parse the input data
lines = input_data.strip().split("\n")
columns = []

# Extract column names and data types
for line in lines:
    col_name, col_type = map(str.strip, line.split("|"))
    hive_type = type_mapping.get(col_type.lower(), "UNKNOWN")  # Default to UNKNOWN if type is not in mapping
    columns.append((col_name, hive_type))

# Output the results
print("Hive Schema:")
for col_name, hive_type in columns:
    print(f"{col_name}, {hive_type}")
